![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [148]:
# Start your code here!
import pandas as pd

# Load the "customer_train.csv" dataset and store it in the ds_jobs and ds_jobs_clean dataframes
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs_clean = ds_jobs
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [149]:
# Print the data types in the ds_jobs_clean dataframe 
ds_jobs_clean.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object

In [150]:
# Convert all columns in the ds_jobs_clean dataframe containing integers to 32-bit integer columns
integer_columns = ["student_id","training_hours","job_change"]
for x in integer_columns:
    ds_jobs_clean[x] = ds_jobs_clean[x].astype("int32")
# Print the data types in the ds_jobs_clean dataframe   
ds_jobs_clean.dtypes

student_id                  int32
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                  int32
dtype: object

In [151]:
# Convert all columns in the ds_jobs_clean dataframe containing float to 16-bit float columns
ds_jobs_clean["city_development_index"] = ds_jobs_clean["city_development_index"].astype("float16")
# Print the data types in the ds_jobs_clean dataframe
ds_jobs_clean.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                  int32
dtype: object

In [152]:
# Convert all remaining columns, all of which are nominal and ordinal categorical columns, into categorical format
for x in ds_jobs_clean.columns:
    if ds_jobs_clean[x].dtype == "object":
        ds_jobs_clean[x] = ds_jobs_clean[x].astype("category")
# Print the data types in the ds_jobs_clean dataframe  
ds_jobs_clean.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience       category
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                   int32
dtype: object

In [153]:
# Identity all ordinal categorical columns in the ds_jobs_clean dataframe and print a list of all unique values in each of those columns
ordinal_category_columns = ["city", "relevant_experience", "enrolled_university", "education_level", "experience", "company_size", "last_new_job"]
for x in ordinal_category_columns:
    print(ds_jobs_clean[x].unique())

['city_103', 'city_40', 'city_21', 'city_115', 'city_162', ..., 'city_121', 'city_129', 'city_8', 'city_31', 'city_171']
Length: 123
Categories (123, object): ['city_1', 'city_10', 'city_100', 'city_101', ..., 'city_94', 'city_97', 'city_98', 'city_99']
['Has relevant experience', 'No relevant experience']
Categories (2, object): ['Has relevant experience', 'No relevant experience']
['no_enrollment', 'Full time course', NaN, 'Part time course']
Categories (3, object): ['Full time course', 'Part time course', 'no_enrollment']
['Graduate', 'Masters', 'High School', NaN, 'Phd', 'Primary School']
Categories (5, object): ['Graduate', 'High School', 'Masters', 'Phd', 'Primary School']
['>20', '15', '5', '<1', '11', ..., '6', '9', '8', '20', NaN]
Length: 23
Categories (22, object): ['1', '10', '11', '12', ..., '8', '9', '<1', '>20']
[NaN, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49', '100-499', '500-999']
Categories (8, object): ['10-49', '100-499', '1000-4999', '10000+', '50-9

In [154]:
# Order each of the ordinal categorical columns of the ds_jobs_clean dataframe into ordered categorical columns
city_categories = ["city_" + str(x) for x in sorted([int(y[5:]) for y in ds_jobs_clean["city"].unique()])]
ds_jobs_clean["city"] = ds_jobs_clean["city"].cat.set_categories(city_categories, ordered=True)

relevant_experience_categories = ["No relevant experience", "Has relevant experience"]
ds_jobs_clean["relevant_experience"] = ds_jobs_clean["relevant_experience"].cat.set_categories(relevant_experience_categories, ordered = True)

enrolled_university_categories = ["no_enrollment", "Primary School", "High School", "Graduate", "Masters", "Phd", "Part time course", "Full time course"]
ds_jobs_clean["enrolled_university"] = ds_jobs_clean["enrolled_university"].cat.set_categories(enrolled_university_categories, ordered = True)

education_level_categories = ["Primary School", "High School", "Graduate", "Masters", "Phd"]
ds_jobs_clean["education_level"] = ds_jobs_clean["education_level"].cat.set_categories(education_level_categories, ordered = True)

experience_categories = ["<1"] + [str(x) for x in range(1,21)] + [">20"]
ds_jobs_clean["experience"] = ds_jobs_clean["experience"].cat.set_categories(experience_categories, ordered = True)

company_size_categories = ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"]
ds_jobs_clean["company_size"] = ds_jobs_clean["company_size"].cat.set_categories(company_size_categories, ordered = True)

last_new_job_categories = ["never", "1", "2", "3", "4", ">4"]
ds_jobs_clean["last_new_job"] = ds_jobs_clean["last_new_job"].cat.set_categories(last_new_job_categories, ordered = True)

In [155]:
# Filter the ds_jobs_clean dataframe so as to contain only those entries for candidates with at least 10 years of experience working in companies with at least 1000 employees and display the head of the result
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean["experience"].isin([str(x) for x in range(10,21)]+[">20"])) & (ds_jobs_clean["company_size"].isin(["1000-4999","5000-9999","10000+"]))]
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
